# o'reillyのネゴザメ言語モデルの本

## 文字列のベクトル化（１）
カウントベースの手法 > 共起頻度アプローチ

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- カウントベースの手法 > 共起頻度アプローチ
  - [共起行列](#共起行列)
  - [コサイン類似度](#コサイン類似度)
  - [類似単語ランキング](#類似単語ランキング)
  - [共起行列をPPMI行列に変換](#共起行列をPPMI行列に変換)
  - [SVDによる次元削減](#SVDによる次元削減)
  - [大規模データセットの次元削減](#大規模データセットの次元削減)
  
## 参考
- https://github.com/oreilly-japan/deep-learning-from-scratch-2/tree/master/ch01
- [言語処理（AI）> ベクトル化 - 開発基盤部会 Wiki](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%EF%BC%88AI%EF%BC%89#k2009143) > [カウントベースの手法 > 共起頻度アプローチ
](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86%EF%BC%88AI%EF%BC%89#o8dc8c2b)

## 環境準備

### インストール

In [ ]:
!pip install numpy
!pip install matplotlib

### インポート

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

## カウントベースの手法 > 共起頻度アプローチ
コーパスの共起関係を行列として表し行列分解の手法を適用。

### 共起行列

In [ ]:
from nekozame.common.util import preprocess, create_co_matrix, cos_similarity

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print('\n corpus:')
print(corpus)
print('\n word_to_id:')
print(word_to_id)
print('\n id_to_word:')
print(id_to_word)

vocab_size = len(word_to_id)

# 共起行列
C = create_co_matrix(corpus, vocab_size)
print('\n 共起行列:')
print(C)

### コサイン類似度

In [ ]:
c0 = C[word_to_id['you']]  #「you」の単語ベクトル
c1 = C[word_to_id['i']]  #「i」の単語ベクトル
print('\n コサイン類似度:')
print(cos_similarity(c0, c1))

### 類似単語ランキング

In [ ]:
from nekozame.common.util import preprocess, create_co_matrix, most_similar

print('\n 類似単語ランキング:')
most_similar('you', word_to_id, id_to_word, C, top=5)

### 共起行列をPPMI行列に変換

In [ ]:
from nekozame.common.util import preprocess, create_co_matrix, cos_similarity, ppmi

np.set_printoptions(precision=3)  # 有効桁3桁で表示

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

# 共起行列
print('co-occurrence matrix')
C = create_co_matrix(corpus, len(word_to_id))
print(C)

print('-'*50)

# PPMI行列
W = ppmi(C)
print('PPMI')
print(W)

### SVDによる次元削減
SVD(特異値分解)  
- SVDでスパース（疎）な大規模行列をデンス（密）な大規模行列に変換
- Uの先頭の次元が重要度が高いのでスライシングで切り捨てる。
- ${X} = {U}{S}{V}^{T}$
  - Sの中の特異値が小さいものは重要度が低い。
  - 小さい特異値を削除することでSの行方向と列方向を削減し、
  - Uを列方向に削減したU'、Vを行方向に削減したV'が作成できる。

In [ ]:
from nekozame.common.util import preprocess, create_co_matrix, ppmi

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id) # 語彙サイズ
window_size = 1 # 共起の最大距離

C = create_co_matrix(corpus, vocab_size, window_size)
W = ppmi(C)

# SVD
U, S, V = np.linalg.svd(W)

np.set_printoptions(precision=3)  # 有効桁３桁で表示

print('counting  co-occurrence  matrix...')
print(C)
print('-'*50)
print('calculating PPMI ...')
print(W)
print('-'*50)
print('calculating SVD ...')
print('U:')
print(U)
print('S:')
print(S)
print('V:')
print(V)

# 先頭２要素をplot
for word, word_id in word_to_id.items():
    plt.annotate(word, (U[word_id, 0], U[word_id, 1]))
plt.scatter(U[:,0], U[:,1], alpha=0.5)
plt.show()

### 大規模データセットの次元削減

#### 大規模なPTBデータセット

In [ ]:
from nekozame.dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
print()
print('corpus size:', len(corpus))
print()
print('id_to_word[0]:', id_to_word[0])
print('id_to_word[1]:', id_to_word[1])
print('id_to_word[2]:', id_to_word[2])
print()
print("word_to_id['car']:", word_to_id['car'])
print("word_to_id['happy']:", word_to_id['happy'])
print("word_to_id['lexus']:", word_to_id['lexus'])

#### LSAによる次元削減
LSA(潜在意味解析)  
- LSAはTruncated SVDとも言う、SVDを使った次元削減法
- 特異値の大きなものに限定して計算することで高速化する。

In [ ]:
from nekozame.common.util import most_similar, create_co_matrix, ppmi
from nekozame.dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')

wordvec_size = 100
vocab_size = len(word_to_id) # 語彙サイズ
window_size = 2 # 共起の最大距離

# 共起行列
print('counting  co-occurrence ...')
C = create_co_matrix(corpus, vocab_size, window_size)
print('-'*50)

# PPMI行列に変換
print('calculating PPMI ...')
W = ppmi(C, verbose=True)
print('-'*50)

# truncated SVDで変換
print('calculating truncated SVD ...')

try:
    # truncated SVD (fast!)
    from sklearn.utils.extmath import randomized_svd
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5,
                             random_state=None)
except ImportError:
    # SVD (slow)
    U, S, V = np.linalg.svd(W)

print(U)
print('-'*50)

# 100次元に削減し類似単語ランキング
print('calculating most similar ...')
word_vecs = U[:, :wordvec_size]
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)